# Database Upload
- Utilize SQLAlchemy to upload the structured data from step 1 into a
Snowflake database.
- Prepare a Python notebook detailing this upload process.

## Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

from sqlalchemy import Boolean, Column, Integer, String

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from dotenv import load_dotenv
import os

## Loading properties from .env

In [2]:
load_dotenv('../config/.env',override=True)

True

In [3]:
def loadenv():
    user = os.getenv("SNOWFLAKE_USER")
    password = os.getenv("SNOWFLAKE_PASSWORD")
    db = os.getenv("SNOWFLAKE_DATABASE")
    account_identifier = os.getenv("SNOWFLAKE_ACCOUNT_IDENTIFIER")
    wh = os.getenv("SNOWFLAKE_WAREHOUSE")
    return user,password ,db ,account_identifier,wh

In [4]:
user , password, db, account_identifier, wh = loadenv()

## Connection to Snowflake

The below function establishes a connection to a Snowflake database. It takes two optional parameters: 
- `path`: specifies the path to a configuration file
- `connection_test`: when set to True, performs a test connection <br>
The function returns a snowflake connection object.

In [5]:
def connectionToSnow(path='../config/.env',connection_test=False):
    load_dotenv(path,override=True)
    user, password, _, account_identifier,_ = loadenv()
    engine = create_engine(
        'snowflake://{user}:{password}@{account_identifier}/'.format(
            user=user,
            password=password,
            account_identifier=account_identifier,
        )
    )
    try:
        connection = engine.connect()
        results = connection.execute('select current_version()').fetchone()
        print(results[0])
        if connection_test:
            connection.close()
        else:
            return connection
    finally:
        engine.dispose()
    

In [6]:
connection = connectionToSnow()

8.5.2


## Utility function execute statements 

In [7]:
def execute(connection,query):
    try:
        results = connection.execute(query)
    except Exception as e:
        print("error-->",e)
    finally:
        print("Done")

## Creating Database

In [ ]:
drop_database_query = '''DROP DATABASE IF EXISTS {};'''.format(db)

create_database_query = "CREATE OR REPLACE DATABASE {};".format(db)

execute(connection, drop_database_query)
execute(connection, create_database_query)

## Creating Warehouse

In [ ]:
create_warehouse = """CREATE OR REPLACE WAREHOUSE {} WITH
   WAREHOUSE_SIZE='X-SMALL'
   AUTO_SUSPEND = 180
   AUTO_RESUME = TRUE
   INITIALLY_SUSPENDED=TRUE;
   """.format(wh)

execute(connection, create_warehouse)

## Creating table to save CSV

In [ ]:
drop_web_data_table = '''DROP TABLE IF EXISTS {};'''.format('CFA_WEB_DATA_R')

create_web_data_table = '''CREATE TABLE {} (
    pdf_link VARCHAR,
    Parent_topic VARCHAR,
    year VARCHAR,
    level VARCHAR,
    Introduction VARCHAR,
    LearningOutcome VARCHAR,
    Summary VARCHAR,
    categories VARCHAR, 
    url VARCHAR,
    topicName VARCHAR
);'''.format('CFA_WEB_DATA_R')

execute(connection, drop_web_data_table)
execute(connection, create_web_data_table)


## Staging CSV 
Uploading CSV to a stage `DUMMY` and then copying it to the table

In [9]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
file_path = parent_directory + "/sample_output/"


create_stage = """CREATE OR REPLACE STAGE DUMMY DIRECTORY = ( ENABLE = true );"""
upload_to_stage  = """PUT file://{}*.csv @{}.public.dummy;""".format(file_path,db)

copy_stage_to_table = """COPY INTO {}
  FROM @{}.public.dummy
  FILE_FORMAT = (type = csv field_optionally_enclosed_by='"')
  PATTERN = 'FinanceHub.csv.gz'
  ON_ERROR = 'skip_file';""".format('CFA_WEB_DATA_R',db,)


execute(connection, create_stage)
execute(connection, upload_to_stage)
execute(connection, copy_stage_to_table)